In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
print("Is gpu available: " + str(torch.cuda.is_available()))

Is gpu available: True


In [3]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 64)
        self.fc2 = nn.Linear(64, 10)  # Assuming 10 classes
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
model = SimpleCNN()

# Transformations for the images
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Loading dataset
train = datasets.ImageFolder('CINIC-10/train', transform=transform)
test = datasets.ImageFolder('CINIC-10/test', transform=transform)

# Creating data loaders
train_loader = DataLoader(train, batch_size=32, shuffle=True)
test_loader = DataLoader(test, batch_size=32, shuffle=False)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    images_processed = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        images_processed += len(images)
        if images_processed % 100 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Processed {images_processed}/{len(train)} images')

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/5, Processed 800/90000 images
Epoch 1/5, Processed 1600/90000 images
Epoch 1/5, Processed 2400/90000 images
Epoch 1/5, Processed 3200/90000 images
Epoch 1/5, Processed 4000/90000 images
Epoch 1/5, Processed 4800/90000 images
Epoch 1/5, Processed 5600/90000 images
Epoch 1/5, Processed 6400/90000 images
Epoch 1/5, Processed 7200/90000 images
Epoch 1/5, Processed 8000/90000 images
Epoch 1/5, Processed 8800/90000 images
Epoch 1/5, Processed 9600/90000 images
Epoch 1/5, Processed 10400/90000 images
Epoch 1/5, Processed 11200/90000 images
Epoch 1/5, Processed 12000/90000 images
Epoch 1/5, Processed 12800/90000 images
Epoch 1/5, Processed 13600/90000 images
Epoch 1/5, Processed 14400/90000 images
Epoch 1/5, Processed 15200/90000 images
Epoch 1/5, Processed 16000/90000 images
Epoch 1/5, Processed 16800/90000 images
Epoch 1/5, Processed 17600/90000 images
Epoch 1/5, Processed 18400/90000 images
Epoch 1/5, Processed 19200/90000 images
Epoch 1/5, Processed 20000/90000 images
Epoch 1/5, Pr

# Test

In [6]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total}%')

Accuracy on test set: 56.577777777777776%


# Single image

In [7]:
from PIL import Image

def predict_single_image(image_path, model, class_names, device):
    # Load the image
    image = Image.open(image_path).convert('RGB')

    # Apply the same transformations as used for the training images
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Transform the image
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Move the image to the same device as the model
    image = image.to(device)

    # Set the model to evaluation mode and get prediction
    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    # Convert the prediction index to a class name
    class_name = class_names[predicted.item()]
    return class_name

# Get class names from the dataset
class_names = test.classes

# Example usage
image_path = 'CINIC-10\\test\\airplane\\cifar10-test-3.png'
predicted_class = predict_single_image(image_path, model, class_names, device)
print(f'Predicted class: {predicted_class}')

Predicted class: airplane
